<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/transfer_learning_from_TADNE_512_tpu_aydao_stylegan2_surgery_model_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-aydao"):
    %cd "/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-aydao
    %cd colab-sg2-aydao
    !git clone --branch model-release https://github.com/aydao/stylegan2-surgery.git
#    !git clone --branch tpu https://github.com/shawwn/stylegan2.git
#    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-surgery
    !mkdir downloads
    !mkdir datasets

/content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery


In [ ]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/sq-512.zip"

!unzip {zip_path} -d /content/

In [ ]:
!gcloud auth login

In [5]:
project_id = 'encoded-phalanx-326615'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
#update this to the path to your image folder
dataset_path = "/content/sq-512"
#give your dataset a name
dataset_name = "sq-512"

#you don't need to edit anything here
#!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}
#!python dataset_tool.py create_from_images gs://train_with_tpu/datasets/{dataset_name} {dataset_path}
!python dataset_tool.py create_aydao gs://train_with_tpu/datasets_aydao {dataset_path} {dataset_name} 512 512


In [6]:
  import os
  import tensorflow.compat.v1 as tf
  tf.disable_eager_execution()
  import pprint
  assert 'COLAB_TPU_ADDR' in os.environ, 'Did you forget to switch to TPU?'
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(tpu_address) as sess:
    devices = sess.list_devices()
  pprint.pprint(devices)
  device_is_tpu = [True if 'TPU' in str(x) else False for x in devices]
  assert True in device_is_tpu, 'Did you forget to switch to TPU?'


[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1181390577653658966),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4295510560114416502),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10574546085292652587),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5655284810999525275),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 5056866136955770771),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15697913543922985507),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1856666063178889126),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 10231963771898090340),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 245156013645842630),
 _DeviceAttributes(/job:tpu_worker/replica:0/

In [ ]:
print(tpu_address)

In [ ]:
#!gsutil cp -r /content/drive/MyDrive/colab-sg2-aydao/stylegan2-surgery/datasets/* gs://train_with_tpu/datasets/


In [ ]:
!gsutil cp -r /content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2/datasets/* gs://train_with_tpu/datasets/


Copying file:///content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2/datasets/sq-512/sq-512-r08.tfrecords [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/drive/MyDrive/colab-sg2-aydao-gpu/stylegan2/datasets/sq-512/sq-512-r07.tfrecords [Content-Type=application/octet-stream]...
Copying file:///content/driv

In [7]:
GCP_PROJECT_ID = 'encoded-phalanx-326615'
PROJECT_NUMBER = '0'


In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gsutil cp -r /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl gs://train_with_tpu/models/

In [ ]:
!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-512 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl

In [ ]:
#!python copy_weights_portable.py /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl ./model.ckpt-1024.pkl

In [ ]:
#!python convert_pkl_to_ckpt.py --ckpt_model_dir /content/drive/MyDrive/models/sq-256 --input_pkl ./network-copyover.pkl
#!python convert_pkl_to_ckpt.py --ckpt_model_dir /content/drive/MyDrive//models/sq-256 --input_pkl /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl


In [ ]:
#!gsutil cp -r /content/drive/MyDrive/models/sq-256/* gs://train_with_tpu/networks_aydao/sq-256

In [ ]:
#!python convert_pkl_to_ckpt.py --tpu_address {tpu_address} --ckpt_model_dir gs://train_with_tpu/models/sq-256 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl

In [ ]:
!chmod 777 -R ./tpu_config.sh
!export TPU_HOST={tpu_address} # placeholder -- refer to shawwn's tpu code
!export TPU_NAME={tpu_address} # placeholder -- refer to shawwn's tpu code
#!export WARM_START_CHECKPOINT_PATH='gs://train_with_tpu/networks_aydao_converted/sq-512'

!./tpu_config.sh

+ python run_training.py --num-gpus=8 --data-dir gs://train_with_tpu/datasets --config=config-f --dataset=sq-512 --gamma=5.0 --mirror-augment=false --metrics=none


attach_debugger <function attach_debugger at 0x7f81e59e7dd0>
print_status <function print_status at 0x7f81e59e7e60>
freeze_forever <function freeze_forever at 0x7f81e59e8050>
quit <function quit at 0x7f81e59e80e0>
save_and_quit <function save_and_quit at 0x7f81e59e8200>
throw_exception <function throw_exception at 0x7f81e59e8290>


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


attach_debugger <function attach_debugger at 0x7f81dd509b00>
print_status <function print_status at 0x7f81dd509b90>
freeze_forever <func

In [ ]:
!python convert_ckpt_to_pkl.py --tpu_address={tpu_address} --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-512 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl

In [ ]:
!gsutil cp -r gs://train_with_tpu/networks_aydao/model.ckpt-9728.pkl /content/drive/MyDrive/networks_aydao/model.ckpt-latest.pkl

Copying gs://train_with_tpu/networks_aydao/model.ckpt-9728.pkl...
/ [1 files][ 1007 MiB/ 1007 MiB]   51.0 MiB/s                                   
Operation completed over 1 objects/1007.6 MiB.                                   


In [ ]:
#!gsutil cp -r gs://train_with_tpu/networks_aydao/model.ckpt-0* /content/drive/MyDrive/networks_aydao/

Copying gs://train_with_tpu/networks_aydao/model.ckpt-0.pkl...
| [1 files][995.8 MiB/995.8 MiB]   65.8 MiB/s                                   
Operation completed over 1 objects/995.8 MiB.                                    


In [ ]:
#!gsutil cp -r gs://train_with_tpu/models/sq-256/model.ckpt-1024* /content/drive/MyDrive/networks_aydao/sq-256/

In [ ]:
#!gsutil cp -r gs://train_with_tpu/networks_aydao/sq-256/* /content/drive/MyDrive/networks_aydao/sq-256/

In [ ]:
#!gsutil cp -r /content/drive/MyDrive/networks_aydao/sq-256/* gs://train_with_tpu/networks_aydao/sq-256/

In [ ]:
#!export TPU_HOST=grpc://10.120.9.58:8470 # placeholder -- refer to shawwn's tpu code
#!export TPU_NAME=grpc://10.120.9.58:8470 # placeholder -- refer to shawwn's tpu code

#!python convert_ckpt_to_pkl.py --tpu_address={tpu_address} --output_pkl_dir gs://train_with_tpu/networks_aydao/ --ckpt_model_dir gs://train_with_tpu/networks_aydao/sq-256 --reference_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl

In [ ]:
#!python convert_ckpt_to_pkl.py --ckpt_model_dir /content/drive/MyDrive/networks_aydao/sq-256 --reference_pkl /content/drive/MyDrive/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl

In [ ]:
#!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-256 --input_pkl gs://train_with_tpu/networks_aydao/model.ckpt-0.pkl

In [ ]:
#!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-256 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-256px-5268480.pkl --kimg 5268480

In [ ]:
#!python convert_pkl_to_ckpt.py --tpu_address={tpu_address} --ckpt_model_dir gs://train_with_tpu/networks_aydao_converted/sq-512 --input_pkl gs://train_with_tpu/models/2020-11-27-aydao-stylegan2ext-danbooru2019s-512px-5268480.pkl --kimg 5268480

In [ ]:
contents of tpu_config.sh
#!/bin/bash
set -e


#
#
#   ~~ aydao tpu config ~~
#   ~~~   aydao 2020   ~~~
#
#   Massive credit to shawwn, gwern, arfa, l4rz
#   tensorfork more broadly, and the "tpu podcast" discord
#
#
#
# This is the mega-long run trained on danbooru2019-s (512px)
#   basis was shawwn's stable tpu branch of the stylegan2 fork
#   https://github.com/shawwn/stylegan2/tree/working
#
# MAJOR CHANGES TO ARCHITECTURE / TRAINING:
#   set G loss to G_logistic_ns (that is, no pathreg)
#   turn off stylemixing, in G_main 
#       style_mixing_prob       = None,
#   increase BOTH latentsize and dlatent size to 1024 
#   normalize_latents set to False
#   only four mapping layers in G_mapping
#   increase fmaps in G:
#       fmap_base           = 32 << 10
#       fmap_max            1024
#   mbstd_group_size    set to 32
#   mbstd_num_features  set to 4
#   set gamma to 5.0 (used to be  --gamma=10.0, lowered at ~4 million)
#   learning rate lowered late in training (see below)
#   topk added at ~2.3 million
#   switched from RANDOM_CROP to TOP_CROP at 4.5 million
#   no noconst (just normal const ... lol I thought it was noconst, oh well)
#   did not use mirror augmentation
#   used figures, portraits, and hand data augmentation (PALM and the rest) 
#       https://www.gwern.net/Crops
#   (data augs were added fairly early in training)
#
#

export LD_LIBRARY_PATH="${LD_LIBRARY_PATH:-/tfk/lib}"

export NOISY=1
export DEBUG=1

config="config-f" # StyleGAN 2

export IMAGENET_UNCONDITIONAL=1
export LABEL_SIZE=0
export LABEL_BIAS=0
export CROP_PADDING=0
# export RANDOM_CROP=1 # Note: this was enabled for the first ~4.5 million 
export TOP_CROP=1 # Note: this was enabled starting at 4.5 million
export G_LR=0.0004 # Note: started at export G_LR=0.002 for first ~3 million
export D_LR=0.0003 # Note: started at export D_LR=0.002 for first ~3 million
export G_LR_MULT=1.0
export D_LR_MULT=1.0

data_dir='gs://train_with_tpu/datasets' # placeholder -- refer to shawwn's tpu code
#data_dir='gs://train_with_tpu/datasets_aydao' # placeholder -- refer to shawwn's tpu code
#dataset=sq-256 # placeholder -- refer to shawwn's tpu code
dataset=sq-512 # placeholder -- refer to shawwn's tpu code
mirror=false # Note: this was false for the entire training
metrics=none

export TPU_SPLIT_COMPILE_AND_EXECUTE=1
export TF_TPU_WATCHDOG_TIMEOUT=1800
#export TPU_HOST=grpc://10.101.159.26:8470 # placeholder -- refer to shawwn's tpu code
#export TPU_NAME=grpc://10.101.159.26:8470 # placeholder -- refer to shawwn's tpu code
cores=8

#export WARM_START_CHECKPOINT_PATH='gs://train_with_tpu/networks_aydao_converted/sq-256'
export WARM_START_CHECKPOINT_PATH='gs://train_with_tpu/networks_aydao_converted/sq-512'

#export IMAGENET_TFRECORD_DATASET='gs://train_with_tpu/datasets/sq-512'
export MODEL_DIR='gs://train_with_tpu/networks_aydao/sq-512'
#export MODEL_DIR='gs://train_with_tpu/networks_aydao/sq-256'
export BATCH_PER=2
#export BATCH_PER=4
#export BATCH_PER=1
export BATCH_SIZE=$(($BATCH_PER * $cores))
#export RESOLUTION=256
export RESOLUTION=512

set -x
python run_training.py --num-gpus="${cores}" --data-dir "${data_dir}" --config="${config}" --dataset="${dataset}" --gamma=5.0 --mirror-augment="${mirror}" --metrics="${metrics}" "$@"